In [ ]:
#作ったモデルを動かしてみる
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()


In [ ]:
model_path="../../models/hf"
model_path="../../models/llama/hf"
model_path="../../models/llama/conv_hf"
model_path="../../models/hf/1code"
model_path="../../models/hf/2jaA"
#model_path= "llm-jp/llm-jp-13b-v1.0"

In [ ]:

model = AutoModelForCausalLM.from_pretrained(model_path,device_map="auto")

In [ ]:
#for name, param in model.named_parameters():
#    print(name)

In [ ]:

tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
tokenizer.pad_token

In [ ]:
t="\n\n### 指示:\n"
t="<SEP>指示<SEP>"
tokenizer.encode(t),tokenizer.tokenize(t)

In [ ]:
t="[###指示]ブラジル"
t="[###指示]元気ですか?"
t="###指示:ブラジル"
t="###指示:東京"
t="###応答:東京"
t="\n\n### 指示:\nブラジル"
t="[###応答]「中古"
t="<SEP> 指示<SEP> この「オーロラ作戦」の記事を踏まえて、攻撃の対象となった企業はどこでしょうか? オーロラ作戦は、"
t="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP> 指示<SEP> この「オーロラ作戦」の記事を踏まえて、攻撃の対象となった企業はどこでしょうか? "
tokenizer.encode(t),tokenizer.tokenize(t)
tokenizer(t)["input_ids"]

In [ ]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=200, 
              repetition_penalty=1.2,
              temperature=0.6,
              )

In [ ]:
import time
text_list=[
"Q. Generate Python code to generate fibonacci sequence.\n A. \n def fibonacci(n):"

]
text_list=[
"llamaのconvertは大変" ,   
"今日はいい",
"富士山は",
"質問: 今日の天気は? 回答:",
"批判的な",
"大規模言語モデルは",
"AI研究の問題点は",
"化学研究の問題点は",
"I have a",
"Functional polymers are",
"機能性高分子は",           
"ホンダ フィット 販売",
"英語: He is a good man. 日本語: ",
]


for text in text_list:
    perp=perplexity(model,tokenizer,text)
    s_time=time.time()
    res=pipe(text)[0]["generated_text"]
    consumed_time=time.time()-s_time
    print("-------")
    print("input: ", text)
    print("perplexity: ",perp)
    print("time: ", consumed_time)
    print("time/character: ", consumed_time/len(res))
    print("output: ",res)

In [ ]:
tokenizer